In [ ]:
# pip install selenium

In [ ]:
# pip install tqdm

In [ ]:
# pip install openpyxl

In [ ]:
# pip install konlpy

In [ ]:
# pip install konlpy

In [ ]:
# pip install OKt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys
from pandas import DataFrame
import time
from tqdm.notebook import tqdm

In [2]:
# URL 설정 및 ChromeDriver 실행
url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000214154&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%A0%84%EC%B2%B4_%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1"
driver = webdriver.Chrome()
time.sleep(2)
driver.maximize_window()
driver.get(url)
time.sleep(5)

In [3]:
temp = driver.find_elements(By.XPATH, '//*[@id="gdasList"]/li[1]/div[2]/div[3]')

print(len(temp))

for i in temp:
    print(i.text)

1
아니 피부 약하고 민감하신분들 이거 진짜 사지마요…..
진심 돈버려요….ㅜㅠ제 피부 꼬라지 좀 보세요 원래 피부 매끈했는데 이거 바르자마자 따갑고 간지럽더니 바른지 하루만에 피부 다 뒤집어졌습니다 돈버리고 피부까지 다 버림..


In [4]:
import re

# CSS Selector로 리뷰 개수 가져오기
num_models_text = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').text.strip()

# 숫자만 추출
num_models = int(re.sub(r"[^\d]", "", num_models_text))  # 숫자만 남김

# 출력
print(f"모델 개수: {num_models}")

# 페이지 수 계산
num_pages = math.ceil(num_models / 10)  # 페이지당 10개로 계산
print(f"페이지 개수: {num_pages}")


모델 개수: 14032
페이지 개수: 1404


In [ ]:
# pip install OKt

In [ ]:
import pandas as pd
import time
from selenium.webdriver.common.by import By

# 데이터를 저장할 리스트
review_data = []

# 상품 리뷰 크롤링 함수
def review_crawling(page_num=10):
    for current_page in range(1, page_num + 1):
        print(f"Scraping page {current_page}...")

        # 리뷰 크롤링
        # 한 페이지에 최대 10개의 리뷰
        for i in range(1, 11):
            try:
                # 리뷰 텍스트 가져오기
                review_xpath = f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner'
                txt = driver.find_element(By.CSS_SELECTOR, review_xpath).text

                # 추가 정보 가져오기
                select1_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span').text
                select_1_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
                select_2_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span').text
                select_2_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
                select_3_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span').text
                select_3_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text

                # # 데이터를 딕셔너리로 저장
                # review_data.append({
                #     "리뷰": txt,
                #     "피부타입": select1_title + ": " + select_1_content,
                #     "피부고민": select_2_title + ": " + select_2_content,
                #     "자극도": select_3_title + ": " + select_3_content
                # })
                
                # 데이터를 텍스트 형식으로 저장
                review_data.append({
                    "리뷰": txt,
                    "피부타입": select_1_content,
                    "피부고민": select_2_content,
                    "자극도": select_3_content
                })
            except Exception as e:
                print(f"Error extracting review {i} on page {current_page}: {e}")

        # 다음 페이지로 이동
        # 마지막 페이지에서는 이동하지 않음
        if current_page < page_num:
            try:
                if current_page % 10 == 0:
                    # 10의 배수일 때 다음 버튼 클릭
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                # 일반 페이지 이동
                else: 
                    next_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page % 10 + 2})')
                next_button.click()
                # 페이지 로딩 대기
                time.sleep(2) 
            except Exception as e:
                print(f"Error moving to page {current_page + 1}: {e}")
                break

    # DataFrame 생성
    df = pd.DataFrame(review_data)
    return df

# 실행
df_review_cos1 = review_crawling(page_num=10)

# 데이터 저장
df_review_cos1.to_csv("reviews_with_details.csv", index=False, encoding="utf-8-sig")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...


In [ ]:
import pandas as pd
# 불용어를 .txt 파일에서 불러오기
with open('data/stopwords.txt', 'r', encoding='cp949') as f:
    # split() 함수를 사용 ,로 구분하여 txt 파일 문자열 구분
    stopwords = set(f.read().split(","))

# 리뷰에서 불용어 제거
def remove_stopwords(review):
    if isinstance(review, str):
        # print("Original review:", review)  # review 출력
        words = review.split()
        # print("Words:", words)  # words 출력
        filtered_words = [word for word in words if word not in stopwords]
        print('filter', filtered_words)
        return ' '.join(filtered_words)
    return review

# 데이터프레임에 불용어 제거 적용
df_review_cos1['불용어 제거 리뷰'] = df_review_cos1['리뷰'].apply(remove_stopwords)

# 수정 후 출력
print(df_review_cos1['불용어 제거 리뷰'])

In [ ]:
df_review_cos1

,리뷰,피부타입,피부고민,자극도,불용어 제거 리뷰
0,아니 피부 약하고 민감하신분들 이거 진짜 사지마요…..\n진심 돈버려요….ㅜㅠ제 피...,복합성에 좋아요,주름/미백에 좋아요,자극이 느껴져요,피부 약하고 민감하신분들 이거 사지마요….. 진심 돈버려요….ㅜㅠ제 피부 꼬라지 보...
1,안녕하세요~솔직한 리뷰만을 남기는 볼탱양입니다.\n제 후기로 여러분들의 선택에 참고...,복합성에 좋아요,진정에 좋아요,자극없이 순해요,안녕하세요~솔직한 리뷰만을 남기는 볼탱양입니다. 후기로 여러분들의 선택에 참고해주셨...
2,안녕하세요!^_^\n항상 솔직한 리뷰만을 적기위해 노력하는 리뷰어 몽몽이이 입니다!...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,안녕하세요!^_^ 항상 솔직한 리뷰만을 적기위해 노력하는 리뷰어 몽몽이이 입니다! ...
3,피부가 너무 뒤집어지고 거칠고 스며들지 않아서 로션들이 ㅠㅠ 세일기간이라 궁금했던 ...,복합성에 좋아요,진정에 좋아요,보통이에요,피부가 뒤집어지고 거칠고 스며들지 않아서 로션들이 세일기간이라 궁금했던 닥터지 사봄...
4,✔️모든 피부 타입 여름에 쓰기 좋은 가벼운 수분크림\n\n✔️산뜻하구 사용감이 가...,복합성에 좋아요,보습에 좋아요,보통이에요,✔️모든 피부 타입 여름에 쓰기 좋은 가벼운 수분크림 ✔️산뜻하구 사용감이 가벼움 ...
...,...,...,...,...,...
95,"닥터지 크림은 워낙 유명하기도 하고,\n성분이 순하고 진정효과가 좋아서\n꾸준히 구...",복합성에 좋아요,진정에 좋아요,자극없이 순해요,"닥터지 크림은 워낙 유명하기도 하고, 성분이 순하고 진정효과가 좋아서 꾸준히 구매해..."
96,4년 넘게 꾸준히 재구매하고 있는 수분크림이 리뉴얼 되었다해서 바로 구매해봤어요 저...,복합성에 좋아요,진정에 좋아요,자극없이 순해요,4년 넘게 꾸준히 재구매하고 있는 수분크림이 리뉴얼 되었다해서 구매해봤어요 저는 민...
97,믿고 쓰는 닥터지 세일하길래 구매했는데 역시나 맘에드네요,건성에 좋아요,보습에 좋아요,자극없이 순해요,믿고 쓰는 닥터지 세일하길래 구매했는데 역시나 맘에드네요
98,"요즘 피부 상태를 먼저 말씀드리면,\n\n1. 일단 예민... (이유 모름ㅠㅠ 온도...",복합성에 좋아요,보습에 좋아요,자극없이 순해요,"요즘 피부 상태를 먼저 말씀드리면, 1. 예민... (이유 모름ㅠㅠ 온도가 높아져서..."


In [18]:
df_review_cos1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   리뷰         100 non-null    object
 1   피부타입       100 non-null    object
 2   피부고민       100 non-null    object
 3   자극도        100 non-null    object
 4   불용어 제거 리뷰  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [ ]:
import pandas as pd
import re

# 특수문자, 이모티콘, 중복 공백 제거 함수
def clean_text(text):
    # 특수문자 및 이모티콘,영어 제거 (한글, 숫자)
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ0-9\s]", "", text)
    # 중복 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 데이터프레임의 모든 컬럼에 적용
for column in df_review_cos1["불용어 제거 리뷰"]:
    df_review_cos1["불용어 제거 리뷰"] = df_review_cos1["불용어 제거 리뷰"].apply(clean_text)

# 결과 출력
df_review_cos1


,리뷰,피부타입,피부고민,자극도,불용어 제거 리뷰
0,아니 피부 약하고 민감하신분들 이거 진짜 사지마요 진심 돈버려요ㅜㅠ제 피부 꼬라지 ...,복합성에 좋아요,주름/미백에 좋아요,자극이 느껴져요,피부 약하고 민감하신분들 이거 사지마요 진심 돈버려요ㅜㅠ제 피부 꼬라지 보세요 피부...
1,안녕하세요솔직한 리뷰만을 남기는 볼탱양입니다 제 후기로 여러분들의 선택에 참고해주셨...,복합성에 좋아요,진정에 좋아요,자극없이 순해요,안녕하세요솔직한 리뷰만을 남기는 볼탱양입니다 후기로 여러분들의 선택에 참고해주셨으면...
2,안녕하세요 항상 솔직한 리뷰만을 적기위해 노력하는 리뷰어 몽몽이이 입니다 오늘도 솔...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,안녕하세요 항상 솔직한 리뷰만을 적기위해 노력하는 리뷰어 몽몽이이 입니다 오늘도 솔...
3,피부가 너무 뒤집어지고 거칠고 스며들지 않아서 로션들이 ㅠㅠ 세일기간이라 궁금했던 ...,복합성에 좋아요,진정에 좋아요,보통이에요,피부가 뒤집어지고 거칠고 스며들지 않아서 로션들이 세일기간이라 궁금했던 닥터지 사봄...
4,모든 피부 타입 여름에 쓰기 좋은 가벼운 수분크림 산뜻하구 사용감이 가벼움 젤타입으...,복합성에 좋아요,보습에 좋아요,보통이에요,모든 피부 타입 여름에 쓰기 좋은 가벼운 수분크림 산뜻하구 사용감이 가벼움 젤타입으...
...,...,...,...,...,...
95,닥터지 크림은 워낙 유명하기도 하고 성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...,복합성에 좋아요,진정에 좋아요,자극없이 순해요,닥터지 크림은 워낙 유명하기도 하고 성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...
96,4년 넘게 꾸준히 재구매하고 있는 수분크림이 리뉴얼 되었다해서 바로 구매해봤어요 저...,복합성에 좋아요,진정에 좋아요,자극없이 순해요,4년 넘게 꾸준히 재구매하고 있는 수분크림이 리뉴얼 되었다해서 구매해봤어요 저는 민...
97,믿고 쓰는 닥터지 세일하길래 구매했는데 역시나 맘에드네요,건성에 좋아요,보습에 좋아요,자극없이 순해요,믿고 쓰는 닥터지 세일하길래 구매했는데 역시나 맘에드네요
98,요즘 피부 상태를 먼저 말씀드리면 1 일단 예민 이유 모름ㅠㅠ 온도가 높아져서인가 ...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,요즘 피부 상태를 먼저 말씀드리면 1 예민 이유 모름ㅠㅠ 온도가 높아져서인가 2 더...


In [ ]:
# import pandas as pd
# from konlpy.tag import Okt

# # Okt 형태소 분석기 생성
# okt = Okt()

# # 토큰화 및 어근 추출 함수
# def tokenize_and_stem(text):
#     if isinstance(text, str):  # 문자열인지 확인
#         # 토큰화
#         tokens = okt.morphs(text)  # 형태소 단위로 토큰화
#         # 어근 추출
#         stems = [okt.normalize(token) for token in tokens]  # Normalize for 어근 추출
#         return stems
#     else:
#         return text  # 문자열이 아니면 그대로 반환

# # 모든 컬럼에 토큰화 및 어근 추출 적용
# for column in df_review_cos1["불용어 제거 리뷰"]:
#     df_review_cos1["불용어 제거 리뷰"] = df_review_cos1["불용어 제거 리뷰"].apply(tokenize_and_stem)

# # 결과 확인
# df_review_cos1


In [ ]:
# import pandas as pd
# from konlpy.tag import Okt


# # Okt 형태소 분석기 생성
# okt = Okt()

# # 토큰화 및 어근 추출 함수
# def tokenize_and_stem(text):
#     if isinstance(text, str):  # 문자열인지 확인
#         # 토큰화
#         tokens = okt.morphs(text)  # 형태소 단위로 토큰화
#         # 어근 추출
#         stems = [okt.normalize(token) for token in tokens]  # Normalize for 어근 추출
#         return stems
#     else:
#         return text  # 문자열이 아니면 그대로 반환

# # 모든 컬럼에 토큰화 및 어근 추출 적용
# for column in df_review_cos1["리뷰"]:
#     df_review_cos1["리뷰"] = df_review_cos1["리뷰"].apply(tokenize_and_stem)

# # 결과
# df_review_cos1

